In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from sklearn.metrics import accuracy_score
from PIL import Image

from efficientnet import EfficientNet

In [2]:
def eval_model(model, dataloader, device, criterion=None):
    loss_value = []
    y_pred = []
    y_true = []

    model.eval()
    with torch.no_grad():
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            if out.size(1) == 1:
                # regression
                out = torch.squeeze(out, 1)

            if criterion is not None:
                loss = criterion(out, yb)
                loss_value.append(loss.item())

            y_pred.append(out.detach().cpu())
            y_true.append(yb.detach().cpu())

    if criterion is not None:
        loss_value = sum(loss_value) / len(loss_value)
        return torch.cat(y_pred), torch.cat(y_true), loss_value
    else:
        return torch.cat(y_pred), torch.cat(y_true)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# EfficientNet model index, i.e. 0 for for EfficientNet-B0
idx = 0
model = EfficientNet(idx, pretrained=True, progress=True)
model.to(device)

val_trainsforms = transforms.Compose([
    transforms.Resize(model.input_spatial_shape[0], interpolation=Image.BICUBIC),
    transforms.CenterCrop(model.input_spatial_shape),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

imagenet_root_path = "imagenet"
# WARNING: following line will download ImageNet validation subset which is ~7GB large
# if imagenet dataset contents are not found in imagenet_root_path, 
# set download=True if you are ok with downloading it
val_dataset = datasets.ImageNet(root=imagenet_root_path, split="val",
                                download=False,
                                transform=val_trainsforms)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=32, shuffle=False,
    num_workers=1, pin_memory=True)

Model weights loaded successfully.


In [4]:
y_pred, y_true = eval_model(model, val_loader, device)
_, y_pred = torch.max(y_pred, 1)

score = accuracy_score(y_pred, y_true)
print("Accuracy: {:.3%}".format(score))

Accuracy: 76.430%


Expected evaluation metric values on ImageNet validation set 

EfficientNet-B0 - 76.43% <br>
EfficientNet-B1 - 78.396% <br>
EfficientNet-B2 - 79.804% <br>
EfficientNet-B3 - 81.542% <br>
EfficientNet-B4 - 83.036% <br>
EfficientNet-B5 - 83.79% <br>
EfficientNet-B6 - 84.136% <br>
EfficientNet-B7 - 84.578% <br>